In [1]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [4]:
import pandas as pd
from tqdm import tqdm
from functools import reduce
from src.query.wdbench import WDBench
from src.query.query_type import QueryType

In [5]:
wdbench = WDBench()

In [7]:
ds = wdbench.queries

In [8]:
ds

,q_id,query_parts,q_type,n_entities,entities
0,1,<http://www.wikidata.org/entity/Q137109> ?x1 ?...,opts,1,[<http://www.wikidata.org/entity/Q137109>]
1,2,?x1 <http://www.wikidata.org/prop/direct/P016>...,opts,1,[<http://www.wikidata.org/entity/Q13414980>]
2,3,?x1 <http://www.wikidata.org/prop/direct/P102>...,opts,1,[<http://www.wikidata.org/entity/Q6721203>]
3,4,?x1 <http://www.wikidata.org/prop/direct/P106>...,opts,4,"[<http://www.wikidata.org/entity/Q1028181>, <h..."
4,5,?x1 <http://www.wikidata.org/prop/direct/P106>...,opts,3,"[<http://www.wikidata.org/entity/Q13382603>, <..."
...,...,...,...,...,...
2653,677,?x1 ?x2 <http://www.wikidata.org/entity/Q31> ....,multiple_bgps,2,"[<http://www.wikidata.org/entity/Q31>, <http:/..."
2654,678,?x1 ?x2 <http://www.wikidata.org/entity/Q48487...,multiple_bgps,1,[<http://www.wikidata.org/entity/Q484876>]
2655,679,?x1 ?x2 <http://www.wikidata.org/entity/Q5> . ...,multiple_bgps,1,[<http://www.wikidata.org/entity/Q5>]
2656,680,?x1 ?x2 <http://www.wikidata.org/entity/Q65364...,multiple_bgps,2,"[<http://www.wikidata.org/entity/Q653645>, <ht..."


In [9]:
ds.rename(columns={'query_parts': 'query'}, inplace=True)

In [10]:
ds.rename(columns={'q_id': 'query_id'}, inplace=True)

In [12]:
ds.rename(columns={'q_type': 'query_type'}, inplace=True)

In [13]:
ds = ds[['query_id', 'query', 'query_type']]

In [14]:
def add_query_to_df(df, nl=False):
    ds_dict = {'query_id': [], 'query': [], 'query_type':[],  'label': []}
    for index, row in tqdm(df.iterrows()):
        filename = f'Q{row["query_id"]}.sparql'

        with open(os.path.join(f'data/raw/wdbench/full_wdbench/{row["query_type"]}', filename), 'r') as file:
            query = file.read()
            query = query.split('NL-QUESTION:')[1 if nl else 0].strip()
                # print(repr(query))
            ds_dict['query'].append(query)
            ds_dict['query_id'].append(row['query_id'])
            ds_dict['query_type'].append(row['query_type'])
            ds_dict['label'].append(-1)
        
    final_ds = pd.DataFrame(ds_dict)
    return final_ds

In [15]:
nl_ds = add_query_to_df(ds, nl=True)

0it [00:00, ?it/s]

2658it [00:00, 8812.32it/s]


In [19]:
nl_ds

,query_id,query,query_type,label
0,1,What information is available about the movie ...,opts,-1
1,2,"What are the family name, date of birth, place...",opts,-1
2,3,"What are the dates of birth, places of birth, ...",opts,-1
3,4,Which painters have a father who is also a pai...,opts,-1
4,5,What are the male ski jumpers from Poland and ...,opts,-1
...,...,...,...,...
2653,677,What are the coordinates of airports located i...,multiple_bgps,-1
2654,678,"What is the image, spouse, and date of birth o...",multiple_bgps,-1
2655,679,Which humans have published in something?,multiple_bgps,-1
2656,680,What films are instances of Ari Folman?,multiple_bgps,-1


In [20]:
nl_ds[['query_id', 'query', 'query_type', 'label']].to_csv('data/raw/wdbench_nl.csv', index=False)

In [21]:
nl_ds['query_len'] = nl_ds.apply(lambda x: len(x['query']), axis=1)

In [22]:
nl_ds['query_len'].describe()

count    2658.000000
mean       74.571859
std        32.258884
min        24.000000
25%        52.000000
50%        68.000000
75%        90.000000
max       294.000000
Name: query_len, dtype: float64